In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0765, -0.0565,  0.0328,  0.1257, -0.0694,  0.0054,  0.1751, -0.0974,
          0.0056,  0.1043],
        [ 0.0237, -0.1911,  0.1564, -0.0314, -0.0103, -0.0017,  0.0606, -0.0023,
          0.1180, -0.0565]], grad_fn=<AddmmBackward>)

In [ ]:
#5.1.1. 自定义块
#在下面的代码片段中，我们从零开始编写一个块。它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。
#注意，下面的MLP类继承了表示块的类。我们的实现将严重依赖父类，只需要提供我们自己的构造函数（Python中的__init__函数）和正向传播函数。
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用`MLP`的父类`Block`的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的正向传播，即如何根据输入`X`返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[-0.1734,  0.1954,  0.0547,  0.0695,  0.1267, -0.1656, -0.0765, -0.0950,
          0.1621, -0.1284],
        [-0.1081,  0.1979,  0.1222,  0.0552,  0.0938, -0.1893, -0.0279, -0.0799,
          0.2117, -0.1701]], grad_fn=<AddmmBackward>)

In [ ]:
#5.1.2. 顺序块
#现在我们可以更仔细地看看Sequential类是如何工作的。回想一下Sequential的设计是为了把其他模块串起来。
#为了构建我们自己的简化的MySequential，我们只需要定义两个关键函数： 
#1. 一种将块逐个追加到列表中的函数。 
#2. 一种正向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。
#下面的MySequential类提供了与默认Sequential类相同的功能。
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 这里，`block`是`Module`子类的一个实例。我们把它保存在'Module'类的成员变量
            # `_children` 中。`block`的类型是OrderedDict。
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.3967, -0.1215, -0.0100,  0.0486, -0.0293,  0.1209, -0.1003,  0.1667,
         -0.1811,  0.1829],
        [-0.3784, -0.0129,  0.1796, -0.0364,  0.0321,  0.1733, -0.0272,  0.2743,
         -0.1667,  0.1450]], grad_fn=<AddmmBackward>)

In [ ]:
#5.1.3. 在正向传播函数中执行代码
#到目前为止，我们网络中的所有操作都对网络的激活值及网络的参数起作用。
#然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项。
#我们称之为常数参数（constant parameters）。例如，我们需要一个计算函数 f(x,w)=c⋅w⊤x 的层，
#其中 x 是输入， w 是我们的参数， c 是某个在优化过程中没有更新的指定常量。因此我们实现了一个FixedHiddenMLP类，如下所示。
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)        

tensor(-0.0869, grad_fn=<SumBackward0>)

In [ ]:
#我们可以混合搭配各种组合块的方法。在下面的例子中，我们以一些想到的的方法嵌套块。
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                      nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1855, grad_fn=<SumBackward0>)